In [241]:
using DataFrames, Plots, Statistics, JSON, Dates, HiGHS #, TuLiPa
include(joinpath(dirname(pwd()),"src/TuLiPa_notmodule.jl"));

Start by changing _common_update_shrinkable!(h, handler, p, start) to print updatemethod (shrink/shift/reset)

    if change <= s.remaining_duration
        println("shrink")
        shrink!(s, p, change)
    elseif (change == (s.remaining_duration + s.minperiod)) && (s.last_shiftperiod != HORIZON_NOSHIFT)
        println("reset_shift")
        reset_shift!(s, p, change)
    else
        println("reset_normal")
        reset_normal!(s, p, change)
    end

### SequentialHorizon

#### Startafter = 0

In [249]:
startafter = Millisecond(0)
shrinkatleast = Millisecond(Hour(4*24))
minperiod = Millisecond(Hour(2*24))
subhorizon = SequentialHorizon(4, Day(4))
hydro_horizon = ShrinkableHorizon(subhorizon, startafter, shrinkatleast, minperiod)
println(hydro_horizon)

t = TwoTime(getisoyearstart(2025), getisoyearstart(1981))
for i in 1:5
    update!(hydro_horizon, t)
#     println(hydro_horizon.subhorizon)
    println(hydro_horizon.handler.shrinker.updates_must)
#     println(hydro_horizon.handler.shrinker.updates_shift)

    t += Day(2)
end

ShrinkableHorizon{SequentialHorizon, SequentialHorizonShrinker}(SequentialHorizon(SequentialPeriods([(1, Millisecond(345600000)), (1, Millisecond(345600000)), (2, Millisecond(345600000))]), nothing), SequentialHorizonShrinker(SequentialPeriodsShrinker(1:2, 1:2, Bool[0, 0], [Millisecond(345600000), Millisecond(345600000)], Millisecond(172800000), [-1, -1, -1, -1], Bool[1, 1, 1, 1], 3, nothing, Millisecond(345600000))))
Bool[1, 1, 1, 1]
shrink
Bool[1, 0, 0, 0]
shrink
Bool[1, 1, 0, 0]
reset_shift
Bool[1, 1, 0, 1]
shrink
Bool[1, 0, 0, 0]


#### Startafter = 2 days

In [251]:
startafter = Millisecond(Day(2))
shrinkatleast = Millisecond(Hour(4*24))
minperiod = Millisecond(Hour(2*24))
subhorizon = SequentialHorizon(5, Day(4))
hydro_horizon = ShrinkableHorizon(subhorizon, startafter, shrinkatleast, minperiod)
println(hydro_horizon)

t = TwoTime(getisoyearstart(2025), getisoyearstart(1981))
for i in 1:5
    update!(hydro_horizon, t)
#     println(hydro_horizon.subhorizon)
    println(hydro_horizon.handler.shrinker.updates_must)
#     println(hydro_horizon.handler.shrinker.updates_shift)

    t += Day(2)
end

ShrinkableHorizon{SequentialHorizon, SequentialHorizonShrinker}(SequentialHorizon(SequentialPeriods([(1, Millisecond(345600000)), (1, Millisecond(345600000)), (1, Millisecond(345600000)), (2, Millisecond(345600000))]), nothing), SequentialHorizonShrinker(SequentialPeriodsShrinker(2:3, 2:3, Bool[0, 0], [Millisecond(345600000), Millisecond(345600000)], Millisecond(172800000), [-1, -1, -1, -1, -1], Bool[1, 1, 1, 1, 1], 4, nothing, Millisecond(345600000))))
Bool[1, 1, 1, 1, 1]
shrink
Bool[1, 1, 0, 0, 0]
shrink
Bool[1, 1, 1, 0, 0]
reset_shift
Bool[1, 1, 1, 0, 1]
shrink
Bool[1, 1, 0, 0, 0]


### AdaptiveHorizon

In [213]:
include("../test/small_model.jl")
data = get_dummy_data()
elements = data["all_elements"]
power_horizon = SequentialHorizon(10, Day(1))
push!(elements, getelement(COMMODITY_CONCEPT, "BaseCommodity", "Power", 
    (HORIZON_CONCEPT, power_horizon)))
addscenariotimeperiod!(elements, "ScenarioTimePeriod", 
    getisoyearstart(1981), 
    getisoyearstart(1983));
modelobjects = getmodelobjects(elements)
modelobjects

Dict{Any, Any} with 4 entries:
  Id("Balance", "PowerBalance_GER") => BaseBalance(Id("Balance", "PowerBalance_…
  Id("Flow", "GenNucGER")           => BaseFlow(Id("Flow", "GenNucGER"), Sequen…
  Id("Flow", "GenCoalGER")          => BaseFlow(Id("Flow", "GenCoalGER"), Seque…
  Id("Flow", "GenBioGER")           => BaseFlow(Id("Flow", "GenBioGER"), Sequen…

#### Startafter = 0

In [214]:
hydroperiodduration = Millisecond(Day(4))
hydroperiods = 4
rhsdata = DynamicRHSAHData("Power")
method = KMeansAHMethod()
clusters = 2
unit_duration = Millisecond(Hour(4))
power_horizon = AdaptiveHorizon(clusters, unit_duration, rhsdata, method, hydroperiods, hydroperiodduration)
power_horizon.data.rhs_terms = modelobjects[Id("Balance", "PowerBalance_GER")].rhsterms

1-element Vector{RHSTerm}:
 BaseRHSTerm(Id("RHSTerm", "DemandGER"), MWToGWhSeriesParam{InfiniteTimeVector{Vector{DateTime}, Vector{Float64}}, RotatingTimeVector{StepRange{DateTime, Millisecond}, SubArray{Float64, 1, Matrix{Float64}, Tuple{UnitRange{Int64}, Int64}, true}}}(InfiniteTimeVector{Vector{DateTime}, Vector{Float64}}([DateTime("2021-01-04T00:00:00"), DateTime("2024-12-30T00:00:00")], [50000.0, 55000.0]), RotatingTimeVector{StepRange{DateTime, Millisecond}, SubArray{Float64, 1, Matrix{Float64}, Tuple{UnitRange{Int64}, Int64}, true}}(DateTime("1980-12-29T00:00:00"):Millisecond(3600000):DateTime("1983-01-03T00:00:00"), [0.75027, 0.72027, 0.70846, 0.7154, 0.74377, 0.81326, 0.96584, 1.08374, 1.14466, 1.16743  …  1.05475, 1.05921, 1.10809, 1.15185, 1.15411, 1.12504, 1.1046, 1.09428, 1.01477, 0.73769], DateTime("1980-12-29T00:00:00"), DateTime("1983-01-03T00:00:00"))), false, Dict{Any, Any}())

In [215]:
startafter = Millisecond(0)
shrinkatleast = Millisecond(Hour(4*24))
minperiod = Millisecond(Hour(2*24))
hydro_horizon = ShrinkableHorizon(power_horizon, startafter, shrinkatleast, minperiod)
println(hydro_horizon.subhorizon.periods)

t = TwoTime(getisoyearstart(2025), getisoyearstart(1981))
for i in 1:5
    update!(hydro_horizon, t)
    for unitstimedelta in hydro_horizon.subhorizon.periods
        println(unitstimedelta.units)
    end
#     println(hydro_horizon.handler.shrinker.updates_must)
#     println(hydro_horizon.handler.shrinker.updates_shift)

    t += Day(2)
end

UnitsTimeDelta[UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000))]
UnitRange{Int64}[1:2, 6:8, 12:14, 18:20, 24:24]
UnitRange{Int64}[3:5, 9:11, 15:17, 21:23]
UnitRange{Int64}[1:2, 6:8, 12:20]
UnitRange{Int64}[3:5, 9:11, 21:24]
UnitRange{Int64}[1:2, 6:8, 12:14, 18:20, 24:24]
UnitRange{Int64}[3:5, 9:11, 15:17, 21:23]
UnitRange{Int64}[1:14, 18:20, 24:24]
UnitRange{Int64}[15:17, 21:23]
shrink
UnitRange{Int64}[1:2, 6:8, 12:12]
UnitRange{Int64}[3:5, 9:11]
UnitRange{Int64}[1:2, 6:8, 12:20]
UnitRange{Int64}[3:5, 9:11, 21:24]
UnitRange{Int64}[1:2, 6:8, 12:14, 18:20, 24:24]
UnitRange{

#### Startafter = 2 days

In [244]:
rhsdata = DynamicRHSAHData("Power")
method = KMeansAHMethod()
clusters = 2
unit_duration = Millisecond(Hour(4))
power_horizon = AdaptiveHorizon(clusters, unit_duration, rhsdata, method, 1, Millisecond(Day(2)), 4, Millisecond(Day(4)))
power_horizon.data.rhs_terms = modelobjects[Id("Balance", "PowerBalance_GER")].rhsterms

1-element Vector{RHSTerm}:
 BaseRHSTerm(Id("RHSTerm", "DemandGER"), MWToGWhSeriesParam{InfiniteTimeVector{Vector{DateTime}, Vector{Float64}}, RotatingTimeVector{StepRange{DateTime, Millisecond}, SubArray{Float64, 1, Matrix{Float64}, Tuple{UnitRange{Int64}, Int64}, true}}}(InfiniteTimeVector{Vector{DateTime}, Vector{Float64}}([DateTime("2021-01-04T00:00:00"), DateTime("2024-12-30T00:00:00")], [50000.0, 55000.0]), RotatingTimeVector{StepRange{DateTime, Millisecond}, SubArray{Float64, 1, Matrix{Float64}, Tuple{UnitRange{Int64}, Int64}, true}}(DateTime("1980-12-29T00:00:00"):Millisecond(3600000):DateTime("1983-01-03T00:00:00"), [0.75027, 0.72027, 0.70846, 0.7154, 0.74377, 0.81326, 0.96584, 1.08374, 1.14466, 1.16743  …  1.05475, 1.05921, 1.10809, 1.15185, 1.15411, 1.12504, 1.1046, 1.09428, 1.01477, 0.73769], DateTime("1980-12-29T00:00:00"), DateTime("1983-01-03T00:00:00"))), false, Dict{Any, Any}())

In [245]:
startafter = Millisecond(Day(2))
shrinkatleast = Millisecond(Hour(4*24))
minperiod = Millisecond(Hour(2*24))
hydro_horizon = ShrinkableHorizon(power_horizon, startafter, shrinkatleast, minperiod)
println(hydro_horizon.subhorizon.periods)

t = TwoTime(getisoyearstart(2025) + Week(3), getisoyearstart(1981) + Week(3))
for i in 1:5
    update!(hydro_horizon, t)
    for unitstimedelta in hydro_horizon.subhorizon.periods
        println(unitstimedelta.units)
    end
#     println(hydro_horizon)
    println(hydro_horizon.handler.shrinker.updates_must)
#     println(hydro_horizon.handler.shrinker.updates_shift)

    t += Day(2)
end

UnitsTimeDelta[UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000)), UnitsTimeDelta(UnitRange{Int64}[], Millisecond(14400000))]
UnitRange{Int64}[1:2, 6:8, 12:12]
UnitRange{Int64}[3:5, 9:11]
UnitRange{Int64}[1:2, 6:8, 12:14, 18:24]
UnitRange{Int64}[3:5, 9:11, 15:17]
UnitRange{Int64}[1:8, 12:14, 18:20, 24:24]
UnitRange{Int64}[9:11, 15:17, 21:23]
UnitRange{Int64}[1:2, 7:8, 12:24]
UnitRange{Int64}[3:6, 9:11]
UnitRange{Int64}[3:6, 9:12, 15:18, 21:24]
UnitRange{Int64}[1:2, 7:8, 13:14, 19:20]
Bool[1, 1, 1, 1, 1]
shrink
1
2
U